In [1]:
from rich.console import Console
from rich_theme_manager import ThemeManager
import pathlib

theme_dir = pathlib.Path("themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

In [2]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [3]:
import gdelt

# get a handle to the gdelt data
gd2 = gdelt.gdelt(version=2)

# get events for today
data = gd2.Search(['2025-08-14'],
    table="events",
    coverage=True
)

# save the results in a CSV file to save time on reruns
data.to_csv("gdelt_results.csv")

here


In [4]:
import pandas as pd

# load the gdelt data
data = (
    pd
    .read_csv('gdelt_results.csv')
    .query('CAMEOCodeDescription.notna()')
    .reset_index(drop=True)
    .to_dict('records')
)

# output a couple samples
console.print(data[:2])

[
    {
        'Unnamed: 0': 0,
        'GLOBALEVENTID': 1257413956,
        'SQLDATE': 20240814,
        'MonthYear': 202408,
        'Year': 2024,
        'FractionDate': 2024.6137,
        'Actor1Code': nan,
        'Actor1Name': nan,
        'Actor1CountryCode': nan,
        'Actor1KnownGroupCode': nan,
        'Actor1EthnicCode': nan,
        'Actor1Religion1Code': nan,
        'Actor1Religion2Code': nan,
        'Actor1Type1Code': nan,
        'Actor1Type2Code': nan,
        'Actor1Type3Code': nan,
        'Actor2Code': 'HLH',
        'Actor2Name': 'HEALTH CARE PROVIDER',
        'Actor2CountryCode': nan,
        'Actor2KnownGroupCode': nan,
        'Actor2EthnicCode': nan,
        'Actor2Religion1Code': nan,
        'Actor2Religion2Code': nan,
        'Actor2Type1Code': 'HLH',
        'Actor2Type2Code': nan,
        'Actor2Type3Code': nan,
        'IsRootEvent': 0,
        'EventCode': 40,
        'CAMEOCodeDescription': ' Consult, not specified below',
        'EventBaseCode': 40,
        'EventRootCode': 4,
        'QuadClass': 1,
        'GoldsteinScale': 1.0,
        'NumMentions': 6,
        'NumSources': 1,
        'NumArticles': 6,
        'AvgTone': 0.62197650310989,
        'Actor1Geo_Type': 0,
        'Actor1Geo_FullName': nan,
        'Actor1Geo_CountryCode': nan,
        'Actor1Geo_ADM1Code': nan,
        'Actor1Geo_ADM2Code': nan,
        'Actor1Geo_Lat': nan,
        'Actor1Geo_Long': nan,
        'Actor1Geo_FeatureID': nan,
        'Actor2Geo_Type': 3,
        'Actor2Geo_FullName': 'Anacostia River, District of Columbia, United States',
        'Actor2Geo_CountryCode': 'US',
        'Actor2Geo_ADM1Code': 'USDC',
        'Actor2Geo_ADM2Code': nan,
        'Actor2Geo_Lat': 38.8537,
        'Actor2Geo_Long': -77.0203,
        'Actor2Geo_FeatureID': '532032',
        'ActionGeo_Type': 3,
        'ActionGeo_FullName': 'Anacostia River, District of Columbia, United States',
        'ActionGeo_CountryCode': 'US',
        'ActionGeo_ADM1Code': 'USDC',
        'ActionGeo_ADM2Code': nan,
        'ActionGeo_Lat': 38.8537,
        'ActionGeo_Long': -77.0203,
        'ActionGeo_FeatureID': '532032',
        'DATEADDED': 20250814013000,
        'SOURCEURL': 
'https://gwhatchet.com/2025/03/03/mfa-hires-new-physicians-for-cedar-hill-hospital-amid-staffing-outreach-concerns/
'
    },
    {
        'Unnamed: 0': 1,
        'GLOBALEVENTID': 1257413957,
        'SQLDATE': 20240814,
        'MonthYear': 202408,
        'Year': 2024,
        'FractionDate': 2024.6137,
        'Actor1Code': 'AUS',
        'Actor1Name': 'SYDNEY',
        'Actor1CountryCode': 'AUS',
        'Actor1KnownGroupCode': nan,
        'Actor1EthnicCode': nan,
        'Actor1Religion1Code': nan,
        'Actor1Religion2Code': nan,
        'Actor1Type1Code': nan,
        'Actor1Type2Code': nan,
        'Actor1Type3Code': nan,
        'Actor2Code': 'USA',
        'Actor2Name': 'HAWAII',
        'Actor2CountryCode': 'USA',
        'Actor2KnownGroupCode': nan,
        'Actor2EthnicCode': nan,
        'Actor2Religion1Code': nan,
        'Actor2Religion2Code': nan,
        'Actor2Type1Code': nan,
        'Actor2Type2Code': nan,
        'Actor2Type3Code': nan,
        'IsRootEvent': 1,
        'EventCode': 42,
        'CAMEOCodeDescription': ' Make a visit',
        'EventBaseCode': 42,
        'EventRootCode': 4,
        'QuadClass': 1,
        'GoldsteinScale': 1.9,
        'NumMentions': 8,
        'NumSources': 1,
        'NumArticles': 8,
        'AvgTone': 0.94161958568738,
        'Actor1Geo_Type': 4,
        'Actor1Geo_FullName': 'Sydney, New South Wales, Australia',
        'Actor1Geo_CountryCode': 'AS',
        'Actor1Geo_ADM1Code': 'AS02',
        'Actor1Geo_ADM2Code': '154637',
        'Actor1Geo_Lat': -33.8833,
        'Actor1Geo_Long': 151.217,
        'Actor1Geo_FeatureID': '-1603135',
        'Actor2Geo_Type': 4,
        'Actor2Geo_FullName': 'Sydney, New South Wales, Australia',
        'Actor2Geo_CountryCode': 'AS',
        'Actor2Geo_ADM1Code

In [6]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

# create the embedding encoder LLM
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# name the collection
collection_name="gdelt"

# create the collection to store the gdelt data
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

In [ ]:
from codetiming import Timer

with Timer(name='qdrant_upload', text="GDELT data points uploaded in {:.2f}s"):
    # load the vector DB
    qdrant.upload_points(
        collection_name=collection_name,
        points=[
            models.PointStruct(
                id=idx,
                vector=encoder.encode(doc["CAMEOCodeDescription"]).tolist(),
                payload=doc
            ) for idx, doc in enumerate(data)
        ]
    )

In [ ]:
# show some stats on the qdrant collection
console.print(qdrant.get_collection(collection_name=collection_name))

In [ ]:
# create a question
user_prompt = "who is making endorsements today?"

# get the vectors for the question
query_vector = encoder.encode(user_prompt).tolist()

# get the results
hits = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3
)

In [ ]:
def format_qdrant_results(results):
    """
    format the results

    :param results:
    :return:
    """
    formatted = []

    # for each result
    for res in results:
        # create a dict of the results
        item = {
            'id': res.id,
            'score': round(res.score, 3),
            'title': res.payload.get('ActionGeo_FullName', 'N/A'),
            'category': res.payload.get('CAMEOCodeDescription', 'N/A'),
            'url': res.payload.get('SOURCEURL', 'N/A'),
            'Actor1Geo_Lat': res.payload.get('Actor1Geo_Lat', 'N/A'),
            'Actor1Geo_Long': res.payload.get('Actor1Geo_Long', 'N/A'),
            'Actor2Geo_Lat': res.payload.get('Actor2Geo_Lat', 'N/A'),
            'Actor2Geo_Long': res.payload.get('Actor2Geo_Long', 'N/A'),
        }

        # add the formatted data to the list
        formatted.append(item)

    # return to the caller
    return formatted

# get the formatted results
formatted_results = format_qdrant_results(hits)

# display the results
for item in formatted_results:
    print(f"ID: {item['id']}, Score: {item['score']}, Title: {item['title']}, Category: {item['category']}, actor 1 lat/long: {item['Actor1Geo_Lat']},{item['Actor1Geo_Long']}, actor 2 lat/long: {item['Actor2Geo_Lat']},{item['Actor2Geo_Long']}, Source url: {item['url']}\n")